In [24]:
import os
import json
import xml.etree.ElementTree as ET
from PIL import Image

# 1. Paths & filenames – adjust these to your setup
images_dir   = r"D:\Deep_Learning\Dataset\MRCNN\val\images"
coco_json_fp = r"D:\Deep_Learning\Dataset\MRCNN\val\images\annotations_val.json"
output_dir   = r"D:\Deep_Learning\Dataset\MRCNN\val\xml_labels"
os.makedirs(output_dir, exist_ok=True)

# 2. Load COCO JSON
with open(coco_json_fp, 'r', encoding='utf-8') as f:
    coco = json.load(f)

# 3. Build helper maps
#    Use only the basename of each file_name so it matches files directly under images_dir
img_map = {
    img['id']: os.path.basename(img['file_name'])
    for img in coco['images']
}

cat_map = {
    cat['id']: cat['name']
    for cat in coco['categories']
}

ann_map = {}
for ann in coco['annotations']:
    ann_map.setdefault(ann['image_id'], []).append(ann)

# 4. Function to create a Pascal VOC XML Element for one image
def make_voc_xml(img_fp, anns):
    with Image.open(img_fp) as img:
        w, h = img.size
        depth = len(img.getbands())

    record = ET.Element("annotation")
    ET.SubElement(record, "folder").text   = os.path.basename(images_dir)
    ET.SubElement(record, "filename").text = os.path.basename(img_fp)

    size = ET.SubElement(record, "size")
    ET.SubElement(size, "width").text  = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text  = str(depth)

    for a in anns:
        x, y, bw, bh = a['bbox']
        xmin = max(0, int(x))
        ymin = max(0, int(y))
        xmax = min(w, int(x + bw))
        ymax = min(h, int(y + bh))
        cls  = cat_map[a['category_id']]

        obj = ET.SubElement(record, "object")
        ET.SubElement(obj, "name").text      = cls
        ET.SubElement(obj, "pose").text      = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"
        bnd = ET.SubElement(obj, "bndbox")
        ET.SubElement(bnd, "xmin").text = str(xmin)
        ET.SubElement(bnd, "ymin").text = str(ymin)
        ET.SubElement(bnd, "xmax").text = str(xmax)
        ET.SubElement(bnd, "ymax").text = str(ymax)

    return record

# 5. Iterate over each image entry in the JSON and write out its VOC XML
for img_id, fname in img_map.items():
    img_fp = os.path.join(images_dir, fname)
    if not os.path.exists(img_fp):
        print(f"[Warning] image missing: {fname}")
        continue

    xml_root = make_voc_xml(img_fp, ann_map.get(img_id, []))
    xml_tree = ET.ElementTree(xml_root)

    xml_filename = os.path.splitext(fname)[0] + ".xml"
    xml_fp = os.path.join(output_dir, xml_filename)
    xml_tree.write(xml_fp, encoding="utf-8", xml_declaration=True)
    print(f"Wrote {xml_filename}")

print("All done!")


Wrote C10.xml
Wrote C1003.xml
Wrote C100_aug.xml
Wrote C1010.xml
Wrote C1019.xml
Wrote C1033.xml
Wrote C1037.xml
Wrote C103_aug.xml
Wrote C1056.xml
Wrote C1086.xml
Wrote C1099.xml
Wrote C1102.xml
Wrote C1106.xml
Wrote C1108.xml
Wrote C1111.xml
Wrote C1117.xml
Wrote C1118.xml
Wrote C1121.xml
Wrote C1124.xml
Wrote C1127.xml
Wrote C1130.xml
Wrote C1131.xml
Wrote C1159.xml
Wrote C115_aug.xml
Wrote C117.xml
Wrote C1176.xml
Wrote C1183.xml
Wrote C118_aug.xml
Wrote C1198.xml
Wrote C1202.xml
Wrote C1203.xml
Wrote C1208.xml
Wrote C120_aug.xml
Wrote C1212.xml
Wrote C1224.xml
Wrote C1233.xml
Wrote C1238.xml
Wrote C1245.xml
Wrote C1257.xml
Wrote C1265.xml
Wrote C1284.xml
Wrote C128_aug.xml
Wrote C1293.xml
Wrote C1297.xml
Wrote C13.xml
Wrote C1301.xml
Wrote C1302.xml
Wrote C1306.xml
Wrote C1317.xml
Wrote C1330.xml
Wrote C1352.xml
Wrote C1355.xml
Wrote C1373.xml
Wrote C1390.xml
Wrote C1393.xml
Wrote C1394.xml
Wrote C1396.xml
Wrote C1399.xml
Wrote C139_aug.xml
Wrote C141_aug.xml
Wrote C142.xml
Wrote 

In [ ]:
#visualize xml output

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageFont

# 1. Adjust these paths
images_dir = r"D:\Deep_Learning\Dataset\MRCNN\train\images"
xml_dir    = r"D:\Deep_Learning\Dataset\MRCNN\train\xml_labels"
output_dir = r"D:\Deep_Learning\Dataset\MRCNN\train\visualized"
os.makedirs(output_dir, exist_ok=True)

# 2. Load a font (fallback to default if Arial isn’t available)
try:
    font = ImageFont.truetype("arial.ttf", size=16)
except IOError:
    font = ImageFont.load_default()

# 3. Loop over all VOC XML files
for xml_fp in glob.glob(os.path.join(xml_dir, "*.xml")):
    tree = ET.parse(xml_fp)
    root = tree.getroot()

    # 4. Find the image filename
    img_name = root.findtext("filename")
    img_path = os.path.join(images_dir, img_name)
    if not os.path.exists(img_path):
        print(f"[Skipping] image not found: {img_name}")
        continue

    # 5. Open image and prepare drawing context
    img = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(img)

    # 6. Draw each bounding box and label
    for obj in root.findall("object"):
        cls = obj.findtext("name")
        bnd = obj.find("bndbox")
        xmin = int(bnd.findtext("xmin"))
        ymin = int(bnd.findtext("ymin"))
        xmax = int(bnd.findtext("xmax"))
        ymax = int(bnd.findtext("ymax"))

        # Draw the rectangle
        draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=2)

        # Measure text size via font.getmask (returns an ImageMask with .size)
        mask = font.getmask(cls)
        text_w, text_h = mask.size

        # Draw a filled rectangle behind the text
        text_bg = [xmin, ymin - text_h, xmin + text_w, ymin]
        draw.rectangle(text_bg, fill="red")

        # Draw the class name
        draw.text((xmin, ymin - text_h), cls, fill="white", font=font)

    # 7. Save the visualized image
    out_path = os.path.join(output_dir, img_name)
    img.save(out_path)
    print(f"Saved visualized: {out_path}")

print("Done! Check the 'visualized' folder for your annotated images.")


In [ ]:
#pascalvocToCSV

In [10]:
import os, glob, csv, xml.etree.ElementTree as ET

def xmls_to_csv(annotations_dir, output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            'label_name',
            'bbox_x1','bbox_y1','bbox_x2','bbox_y2',
            'image_name',
            'image_width','image_height'
        ])
        for xml_fp in glob.glob(os.path.join(annotations_dir, '*.xml')):
            tree = ET.parse(xml_fp)
            root = tree.getroot()
            image_name = root.findtext('filename')
            size = root.find('size')
            w, h = size.findtext('width'), size.findtext('height')
            for obj in root.findall('object'):
                label = obj.findtext('name')
                b = obj.find('bndbox')
                writer.writerow([
                    label,
                    b.findtext('xmin'),
                    b.findtext('ymin'),
                    b.findtext('xmax'),
                    b.findtext('ymax'),
                    image_name,
                    w, h
                ])
    print(f"Wrote CSV to {output_csv}")

# Usage
xmls_to_csv(
  annotations_dir=r"D:\Deep_Learning\Dataset\MRCNN\train\xml_labels",
  output_csv=r"D:\Deep_Learning\Dataset\MRCNN\train\annotations_final.csv"
)


Wrote CSV to D:\Deep_Learning\Dataset\MRCNN\train\annotations_final.csv


In [1]:
import os, glob, csv, xml.etree.ElementTree as ET

def xmls_to_csv(annotations_dir, output_csv):
    """
    Convert Pascal VOC XMLs → CSV, including one “no_defect” row
    for images with zero objects.
    """
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            'label_name',
            'bbox_x1','bbox_y1','bbox_x2','bbox_y2',
            'image_name',
            'image_width','image_height'
        ])

        for xml_fp in glob.glob(os.path.join(annotations_dir, '*.xml')):
            tree = ET.parse(xml_fp)
            root = tree.getroot()

            image_name = root.findtext('filename')
            size       = root.find('size')
            w, h       = size.findtext('width'), size.findtext('height')

            objects = root.findall('object')
            if not objects:
                # No objects → emit a “negative” row
                writer.writerow([
                    'No_defect',   # label
                    0, 0, 0, 0,    # dummy bbox
                    image_name,
                    w, h
                ])
            else:
                for obj in objects:
                    label = obj.findtext('name')
                    b     = obj.find('bndbox')
                    xmin  = b.findtext('xmin')
                    ymin  = b.findtext('ymin')
                    xmax  = b.findtext('xmax')
                    ymax  = b.findtext('ymax')

                    writer.writerow([
                        label,
                        xmin, ymin, xmax, ymax,
                        image_name,
                        w, h
                    ])

    print(f"Wrote CSV (with negatives) to {output_csv}")

# Usage
xmls_to_csv(
    annotations_dir=r"D:\Deep_Learning\Dataset\MRCNN\train\xml_labels",
    output_csv    =r"D:\Deep_Learning\Dataset\MRCNN\train\annotations_final1.csv"
)


Wrote CSV (with negatives) to D:\Deep_Learning\Dataset\MRCNN\train\annotations_final1.csv


In [27]:
#visuall check CSV
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# === CONFIGURATION – adjust these paths to your setup ===
csv_fp         = r"D:\Deep_Learning\Dataset\MRCNN\val\annotations_final1.csv"
images_dir     = r"D:\Deep_Learning\Dataset\MRCNN\val\images"
output_png     = r"D:\Deep_Learning\Dataset\MRCNN\val\visual_check.png"
samples_per_cat = 5
# ========================================================

# 1. Load and normalize the CSV
df = pd.read_csv(csv_fp)
df['label_name'] = df['label_name'].astype(str).str.strip()

# 2. Derive categories from the data, then include true negatives
cats_in_data = sorted(df['label_name'].unique().tolist())
if 'no_defect' not in cats_in_data:
    cats_in_data.append('no_defect')
print("Using categories:", cats_in_data)

# 3. Prepare figure
fig, axes = plt.subplots(len(cats_in_data), samples_per_cat, figsize=(15, 9))

# 4. List all images on disk
all_images = [
    f for f in os.listdir(images_dir)
    if f.lower().endswith(('.jpg','.jpeg','.png'))
]

# 5. For each category, sample and plot
for i, cat in enumerate(cats_in_data):
    if cat != 'no_defect':
        # images with this label
        img_list = df.loc[df['label_name'] == cat, 'image_name'].unique().tolist()
    else:
        # true negatives
        annotated = set(df['image_name'].unique())
        img_list = [f for f in all_images if f not in annotated]

    if not img_list:
        print(f"[!] No images found for '{cat}'")
        for j in range(samples_per_cat):
            axes[i, j].axis('off')
        continue

    sampled = random.sample(img_list, min(samples_per_cat, len(img_list)))
    print(f"Category '{cat}': sampling {len(sampled)} → {sampled}")

    for j in range(samples_per_cat):
        ax = axes[i, j]
        if j >= len(sampled):
            ax.axis('off')
            continue

        img_name = sampled[j]
        img_path = os.path.join(images_dir, img_name)
        if not os.path.exists(img_path):
            ax.axis('off')
            ax.set_title(f"Missing: {img_name}", fontsize=8)
            continue

        # display image
        img = Image.open(img_path).convert("RGB")
        ax.imshow(img)
        ax.axis('off')
        if j == 0:
            ax.set_ylabel(cat, fontsize=12)

        # draw bboxes (skip for no_defect)
        if cat != 'no_defect':
            # choose color by category
            color = 'green' if cat.lower() == 'crack' else 'red'
            subset = df[(df['image_name'] == img_name) & (df['label_name'] == cat)]
            for _, row in subset.iterrows():
                xmin, ymin = row['bbox_x1'], row['bbox_y1']
                xmax, ymax = row['bbox_x2'], row['bbox_y2']
                rect = patches.Rectangle(
                    (xmin, ymin),
                    xmax - xmin,
                    ymax - ymin,
                    fill=False,
                    linewidth=2,
                    edgecolor=color
                )
                ax.add_patch(rect)

# 6. Save out
plt.tight_layout()
plt.savefig(output_png, dpi=150)
plt.close(fig)

print(f"✅ Visual check saved to: {output_png}")


Using categories: ['Crack', 'Delamination', 'No_defect', 'no_defect']
Category 'Crack': sampling 5 → ['C200.png', 'C482.jpg', 'C115_aug.jpg', 'C1086.jpg', 'C128_aug.jpg']
Category 'Delamination': sampling 5 → ['D420.jpg', 'D726.jpg', 'D3_aug2.jpg', 'D111_aug1.jpg', 'D104.jpg']
Category 'No_defect': sampling 5 → ['Nodefect__839.jpg', 'Nodefect__762.jpg', 'Nodefect__18.jpg', 'Nodefect__763.jpg', 'Nodefect__770.jpg']
[!] No images found for 'no_defect'
✅ Visual check saved to: D:\Deep_Learning\Dataset\MRCNN\val\visual_check.png


In [19]:
import pandas as pd

# 1) Point this to your actual CSV location
csv_fp = r"D:\Deep_Learning\Dataset\MRCNN\train\annotations_final1.csv"

# 2) Load and print unique labels
df = pd.read_csv(csv_fp)
print("Unique label_name values in CSV:")
print(df['label_name'].unique())


Unique label_name values in CSV:
['Crack' 'Delamination' 'No_defect']
